---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [13]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def blight_model():
    
    from sklearn.utils import resample
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.preprocessing import StandardScaler,MinMaxScaler 
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import auc
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_curve, auc
    
    # load datasets
    df_train = pd.read_csv('./train.csv',encoding = "ISO-8859-1")
    df_test = pd.read_csv('./test.csv',encoding = "ISO-8859-1")
    df_latlong = pd.read_csv('./latlons.csv',encoding = "ISO-8859-1")
    df_add = pd.read_csv('./addresses.csv',encoding = "ISO-8859-1")
    
    # remove empty features
    to_delete = ['violation_zip_code','non_us_str_code','grafitti_status']
    df_train = df_train.drop(to_delete,axis = 1)
    df_test = df_test.drop(to_delete,axis = 1)
    
    # remove features to prevend data leakage
    to_delete = ['compliance_detail','payment_amount','payment_date','payment_status',
                'balance_due','collection_status','compliance_detail']
    df_train = df_train.drop(to_delete,axis = 1)
    
    # drop missing values - compliance
    df_train = df_train[df_train['compliance'].notnull()]
    
    # rebalance samples
    df_majority = df_train[df_train['compliance']==0]
    df_minority = df_train[df_train['compliance']==1]
    
    # upsample minority class
    df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=df_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results
    
    # combine minority class with downsampled majority class
    df_train = pd.concat([df_majority, df_minority_upsampled])
    
    # set ticket id as index
    df_train = df_train.set_index('ticket_id')
    df_test = df_test.set_index('ticket_id')
    
    # simplify the content of the agency name using initials
    agencies = {'Buildings, Safety Engineering & Env Department':'BSEED',
               'Department of Public Works':'DPW',
               'Health Department':'HD',
               'Detroit Police Department':'DPD',
               'Neighborhood City Halls':'NCH'
               }

    df_train['agency_name'] = df_train['agency_name'].replace(agencies)
    df_test['agency_name'] = df_test['agency_name'].replace(agencies)
    
    # inspectors
    # format the names as aaaa, bbbbb
    df_train['inspector_name'] = df_train['inspector_name'].str.lower()
    df_test['inspector_name'] = df_test['inspector_name'].str.lower()
    
    # bins by count of tickets
    df_train['active_inspector'] = df_train['inspector_name'].replace(df_train['inspector_name'].value_counts().to_dict())
    df_test['active_inspector'] = df_test['inspector_name'].replace(df_test['inspector_name'].value_counts().to_dict())
    
    df_train['active_inspector_bins'], bins_train = pd.qcut(df_train['active_inspector'],q=6,
                                                       labels=['Inactive','Very_Low','Low','Average','High','Very_High'],retbins=True)
    bins_train[0] = 0.000
    
    df_test['active_inspector_bins'] = pd.cut(df_test['active_inspector'],bins=bins_train,
                                          labels=['Inactive','Very_Low','Low','Average','High','Very_High'],right=True)
    
    # drop inspector name
    df_train = df_train.drop(['inspector_name'],axis=1)
    df_test = df_test.drop(['inspector_name'],axis=1)
    
    # drop violator name
    df_train = df_train.drop(['violator_name'],axis=1)
    df_test = df_test.drop(['violator_name'],axis=1)
    
    # create new feature: is_home where is_home is True if violation_street_name==mailing_address_str_name
    df_train['is_home'] = df_train['violation_street_name']==df_train['mailing_address_str_name']
    df_test['is_home'] = df_test['violation_street_name']==df_test['mailing_address_str_name'] 
    
    # create new feature: has_address
    df_train['has_address'] = ~df_train['mailing_address_str_number'].isnull()
    df_test['has_address'] = ~df_test['mailing_address_str_number'].isnull()
    
    # drop mailing address
    df_train = df_train.drop(['mailing_address_str_number'],axis=1)
    df_test = df_test.drop(['mailing_address_str_number'],axis=1)
    
    # drop street number
    df_train = df_train.drop(['mailing_address_str_name'],axis=1)
    df_test = df_test.drop(['mailing_address_str_name'],axis=1)
    
    # new features
    # is local (Michican)
    # is neighbor (Wisconsin, Illinois, Indiana, Ohio)
    # is foreign
    
    df_train['is_local'] = df_train['state'] == 'MI'
    df_test['is_local'] = df_test['state'] == 'MI'
    
    neighbor_state = ['WI','IL','IN','OH']
    df_train['is_neigh'] = df_train['state'].isin(neighbor_state)
    df_test['is_neigh'] = df_test['state'].isin(neighbor_state)
    
    df_train['is_foreign'] = df_train['country']!='USA'
    df_test['is_foreign'] = df_test['country']!='USA'
    
    # Set missing state for foreign country
    df_train.loc[df_train['state'].isnull(),'state']='XX'
    df_test.loc[df_test['state'].isnull(),'state']='XX'
    
    # format city names
    df_train['city'] = df_train['city'].str.lower()
    df_test['city'] = df_test['city'].str.lower()
    
    # find missing city for df_test
    df_test.loc[df_test['city'].isnull(),'city']='westerville'
    
    # transform dates
    df_train['ticket_issued_date'] = pd.to_datetime(df_train['ticket_issued_date'])
    df_test['ticket_issued_date'] = pd.to_datetime(df_test['ticket_issued_date'])
    
    df_train['ticket_issued_month'] = df_train['ticket_issued_date'].dt.month
    df_test['ticket_issued_month'] = df_test['ticket_issued_date'].dt.month
    
    df_train['ticket_issued_start'] = df_train['ticket_issued_date'].dt.day<=15
    df_test['ticket_issued_start'] = df_test['ticket_issued_date'].dt.day<=15
    
    # bin disposition
    disposition_bins = [
        'Responsible by Default',
        'Responsible by Admission',
        'Responsible by Determination',
        'Responsible (Fine Waived) by Deter']
    
    df_train['disposition_bins'] = df_train['disposition'].apply(lambda x: x if x in disposition_bins else 'Other')
    df_test['disposition_bins'] = df_test['disposition'].apply(lambda x: x if x in disposition_bins else 'Other')
    
    # has hearing
    df_train['has_hearing'] = ~df_train['hearing_date'].isnull()
    df_test['has_hearing'] = ~df_test['hearing_date'].isnull()
    
    df_train['violation_code'] = (df_train['violation_code'].str.replace('(','')).str.replace(')','').str.replace(' ','').str.replace('-','').str.extract('(\d+)',expand=True).apply(lambda x: x[0:8])
    df_test['violation_code'] = (df_train['violation_code'].str.replace('(','')).str.replace(')','').str.replace(' ','').str.replace('-','').str.extract('(\d+)',expand=True).apply(lambda x: x[0:8])
    
    to_drop = ['violation_street_number','zip_code',
               'hearing_date','ticket_issued_date',
               'violation_description','active_inspector',
              'city','country','violation_street_name',
              'disposition']
    df_train = df_train.drop(to_drop,axis=1)
    df_test = df_test.drop(to_drop,axis=1)
    
    # set index
    df_add = df_add.set_index('ticket_id')

    # combine with data
    df_train = pd.merge(left=df_train,right=df_add,how='left',left_index=True,right_index=True)
    df_test = pd.merge(left=df_test,right=df_add,how='left',left_index=True,right_index=True)
    
    # set index
    df_latlong = df_latlong.set_index('address')

    # combine with data
    df_train = pd.merge(left=df_train,right=df_latlong,how='left',left_on='address',right_index=True)
    df_test = pd.merge(left=df_test,right=df_latlong,how='left',left_on='address',right_index=True)

    # drop address
    df_train = df_train.drop(['address'],axis=1)
    df_test = df_test.drop(['address'],axis=1)
    
    step = 0.03
    to_bin = lambda x: str(np.floor(x / step) * step)
    df_train["latbin"] = df_train['lat'].map(to_bin)
    df_train["lonbin"] = df_train['lon'].map(to_bin)

    df_test["latbin"] = df_test['lat'].map(to_bin)
    df_test["lonbin"] = df_test['lon'].map(to_bin)
    
    df_train_dum = pd.get_dummies(data=df_train,
                              columns=['agency_name','state','disposition_bins',
                                      'active_inspector_bins','ticket_issued_month',
                                       'ticket_issued_start','violation_code','latbin','lonbin'],
                              drop_first=False)

    df_test_dum = pd.get_dummies(data=df_test,
                              columns=['agency_name','state','disposition_bins',
                                      'active_inspector_bins','ticket_issued_month',
                                       'ticket_issued_start','violation_code','latbin','lonbin'],
                              drop_first=False)

    # drop column differences
    df_test = df_test_dum.drop(['disposition_bins_Other','lonbin_-82.89','lat','lon'],axis=1)

    df_train = df_train_dum.drop(['agency_name_HD', 'agency_name_NCH', 'compliance',
                                  'latbin_42.12', 'latbin_42.15', 'latbin_42.51',
                                  'latbin_42.57', 'latbin_42.6', 'latbin_42.63',
                                  'latbin_42.66', 'latbin_42.69', 'latbin_42.72',
                                  'latbin_42.75', 'latbin_42.9', 'latbin_44.76',
                                  'lonbin_-82.53', 'lonbin_-83.34', 'lonbin_-83.46',
                                  'lonbin_-83.49', 'lonbin_-83.55', 'lonbin_-83.76',
                                  'lonbin_-84.12', 'lonbin_-84.42', 'state_BL',
                                  'violation_code_6163', 'violation_code_9136',
                                  'compliance','lat','lon'],axis=1)

    Y_train = df_train_dum['compliance']

    X_train = df_train.copy()
    X_test = df_test.copy()

    stdsc = StandardScaler().fit(X_train)
    X_train = stdsc.transform(X_train)
    X_test = stdsc.transform(X_test)

    X_train, X_val, y_train, y_val = train_test_split(X_train,Y_train)


    rdf = RandomForestClassifier(n_estimators=200,oob_score = True)
   
    rdf.fit(X_train,y_train)

    y_score_rdf = rdf.predict_proba(X_val)
    
    y_test_proba = rdf.predict_proba(X_test)[:,1]
    
    result = pd.Series(y_test_proba,index=df_test.index)
    
    return result